#### Monday, April 15, 2024

mamba activate langchain3

This all runs locally. Some cells had to be re-run to get the right output.

In [3]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/NexusRaven-V2-13B-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Sure, I'll do my best to introduce myself. My name is LLaMA, and I'm a large language model trained by a team of researcher at Meta AI. My capabilities include answering questions, generating text, translating languages, and more. I'm constantly learning and improving, so please bear with me if I make any mistakes. How can I help you today?", role='assistant', function_call=None, tool_calls=None)


# Composability

This notebook shows off some basic functionality around LangChain Expression Language, which makes it really easy to compose arbitrary chains.

For a much deeper dive, see:

- [Full LangChain Expression Language Documentation](https://python.langchain.com/docs/expression_language/)

## Basic Composability

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

In [4]:

# model = ChatOpenAI()
model = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [5]:

output_parser = StrOutputParser()

In [6]:
chain = prompt | model | output_parser

In [7]:
chain.invoke({"topic": "bears"})

"Sure, here's a joke: Why did the bear go to the doctor? Because he was feeling ruff!"

### Batching

In [8]:
chain.batch([{"topic": "bear"}, {"topic": "clowns"}])

["Here's a joke for you: Why did the bear go to the doctor? Because he had a hiccup!",
 "Okay, here's one: Why did the clown break up with his girlfriend? Because he knew it was a joke!"]

### Streaming

In [9]:
for s in chain.stream({"topic": "bears"}):
    print(s)

S
ure
!
 Here
'
s
 one
:
 Why
 did
 the
 bear
 go
 to
 the
 doctor
?
 Because
 he
 had
 a
 hair
-
ra
ising
 experience
!



## RunnablePassthrough

In [10]:
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

In [11]:
chain = prompt | model | output_parser

In [12]:
question = "what is langsmith"
context = "langsmith is a testing and observability platform built by the langchain team"
chain.invoke({"question": question, "context": context})

'Langsmith is a testing and observability platform built by the langchain team. It provides a comprehensive set of tools for testing and observing microservices, allowing developers to quickly and easily test and debug their applications. The platform includes features such as service discovery, circuit breakers, and distributed tracing, making it an ideal choice for teams looking to improve the quality of their microservices.'

In [13]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever
) | chain

In [14]:
retrieval_chain.invoke({"question": "what is langsmith"})

# 31.9s

' \nCall: initialize_agent(tools=[\'serpapi\', \'llm-math\'], llms=[\'LLM-90\'], agent=\'LangSmith\')<bot_end> \nThought: The function call `initialize_agent(tools=[\'serpapi\', \'llm-math\'], llms=[\'LLM-90\'], agent=\'LangSmith\')` answers the question "What is Langsmith?" by initializing an agent called "LangSmith" with a set of tools (in this case, "serpapi" and "llm-math") and a set of LLMs (in this case, "LLM-90").\n\nThe `initialize_agent` function is used to initialize an agent in the LangChain ecosystem. An agent is a program that can generate responses based on descriptions, and it can be trained using various tools and LLMs.\n\nBy calling `initialize_agent` with the specified parameters, we are initializing an agent called "LangSmith" that can use the "serpapi" and "llm-math" tools to generate responses based on the "LLM-90" LLM. This means that when the "LangSmith" agent is asked a question, it will use the "serpapi" and "llm-math" tools to generate a response based on the 

## RunnableParallel

In [15]:
from langchain_core.runnables import RunnableParallel

In [16]:
prompt = ChatPromptTemplate.from_template("""{question}""")
simple_chain = prompt | model | output_parser

In [17]:
parallel_chain = RunnableParallel({
    "retrieved_answer": retrieval_chain,
    "simple_answer": simple_chain
})

In [19]:
parallel_chain.invoke({"question": "what is langsmith"})

# 7m 50.3s .... stopped the server because it was outputting gibberish ...

# 6.8s .... then starting up the server, then resubmitting this cell, it worked! ... 

{'retrieved_answer': '',
 'simple_answer': 'Langsmith is a company that specializes in creating and selling language learning software. It offers a wide range of products, including language learning apps for both iOS and Android devices, as well as online courses and other educational resources.'}

In [20]:
for s in parallel_chain.stream({"question": "what is langsmith"}):
    print(s)

{'simple_answer': 'The'}
{'simple_answer': ' word'}
{'simple_answer': ' "'}
{'simple_answer': 'lang'}
{'simple_answer': 'sm'}
{'simple_answer': 'ith'}
{'simple_answer': '"'}
{'simple_answer': ' is'}
{'simple_answer': ' an'}
{'simple_answer': ' arch'}
{'simple_answer': 'a'}
{'simple_answer': 'ic'}
{'simple_answer': ' term'}
{'simple_answer': ' for'}
{'simple_answer': ' a'}
{'simple_answer': ' black'}
{'simple_answer': 'sm'}
{'simple_answer': 'ith'}
{'simple_answer': ' who'}
{'simple_answer': ' special'}
{'simple_answer': 'izes'}
{'simple_answer': ' in'}
{'simple_answer': ' making'}
{'simple_answer': ' tools'}
{'simple_answer': ' and'}
{'simple_answer': ' weapons'}
{'simple_answer': ' out'}
{'simple_answer': ' of'}
{'simple_answer': ' iron'}
{'simple_answer': ' and'}
{'simple_answer': ' steel'}
{'simple_answer': '.'}
{'simple_answer': ' The'}
{'simple_answer': ' term'}
{'simple_answer': ' comes'}
{'simple_answer': ' from'}
{'simple_answer': ' the'}
{'simple_answer': ' Old'}
{'simple_answ

In [21]:
result = {}
for s in parallel_chain.stream({"question": "what is langsmith"}):
    for k,v in s.items():
        if k not in result:
            result[k] = ""
        result[k] += v
    print(result)

# 32.4s

{'simple_answer': 'The'}
{'simple_answer': 'The term'}
{'simple_answer': 'The term "'}
{'simple_answer': 'The term "lang'}
{'simple_answer': 'The term "langsm'}
{'simple_answer': 'The term "langsmith'}
{'simple_answer': 'The term "langsmith"'}
{'simple_answer': 'The term "langsmith" refers'}
{'simple_answer': 'The term "langsmith" refers to'}
{'simple_answer': 'The term "langsmith" refers to a'}
{'simple_answer': 'The term "langsmith" refers to a person'}
{'simple_answer': 'The term "langsmith" refers to a person who'}
{'simple_answer': 'The term "langsmith" refers to a person who is'}
{'simple_answer': 'The term "langsmith" refers to a person who is sk'}
{'simple_answer': 'The term "langsmith" refers to a person who is skilled'}
{'simple_answer': 'The term "langsmith" refers to a person who is skilled in'}
{'simple_answer': 'The term "langsmith" refers to a person who is skilled in languages'}
{'simple_answer': 'The term "langsmith" refers to a person who is skilled in languages,'}
{'